In [1]:
%matplotlib notebook
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tensorflow import keras
from tensorflow import set_random_seed
from numpy.random import seed
from datetime import datetime

# set numpy seed for reproducibility
seed(1)
# set tf seed for reproducibility
set_random_seed(2)
#
# Turn off GPU usage for tf
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
#

In [2]:
# Read the previously saved dataframe
df_covtype_ohe = pd.read_csv(os.path.join(os.getcwd(), 'covtype_ohe.csv'), dtype=np.float32)

In [3]:
#df_covtype_ohe.head(2)

In [4]:
def train_test_split(df_covtype_ohe):
    '''
    Split the one hot encoded dataset onto training set and test set
    according to UCI's repository guidelines
    '''
    # First 15120 rows for the training set
    X_train = df_covtype_ohe[:15120].copy()
    # The last seven colums are the targets
    X_train, y_train = X_train.iloc[:, :51], X_train.iloc[:, 51:]
    # The remaining rows are for the test set
    X_test = df_covtype_ohe[15121:].copy()
    X_test, y_test = X_test.iloc[:, :51], X_test.iloc[:, 51:]
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split(df_covtype_ohe)

In [5]:
# Check shapes consistency
print(f'X_train: {X_train.shape}, X_test: {X_test.shape}, ' \
      f'y_train: {y_train.shape}, y_test: {y_test.shape}')

X_train: (15120, 51), X_test: (565891, 51), y_train: (15120, 7), y_test: (565891, 7)


In [6]:
# Let's standardize the training set and test set.
# Note that we use the training set ONLY to calculate the mean and standard deviation
# then normalize the training set 
# and finally use the (training) mean and standard deviation to normalize the test set.
# This ensures no data leakage.

def train_test_normalize(X_train, X_test):
    '''
    Perform standardization on the training set and transforms the
    test set accordingly
    '''
    # The numerical columns we want to normalize
    numerical_columns = ['Elevation',
                         'Distance_To_Hydrology',
                         'Horizontal_Distance_To_Roadways',
                         'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
                         'Horizontal_Distance_To_Fire_Points']
    # Calculate the mean and standard deviation of the training set
    X_train_num_cols_mean = X_train[numerical_columns].mean()
    X_train_num_cols_std = X_train[numerical_columns].std()
    # Perform standardization over the numerical columns of the training set
    X_train_std = (X_train[numerical_columns] - X_train_num_cols_mean) / X_train_num_cols_std
    # Concatenate side-by-side the normalized training set and the one-hot encoded features
    # Note that we index X_train dataframe by the (set) difference of the overall features
    # minus the numerical ones
    ohe_features = X_train.columns.difference(other=numerical_columns, sort=False)
    X_train_std = pd.concat([X_train_std, X_train[ohe_features]], axis=1)
    # Perform standardization over the numerical columns of the test set, using the mean and std
    # of the training set as discussed earlier
    X_test_std = (X_test[numerical_columns] - X_train_num_cols_mean) / X_train_num_cols_std
    # Concatenate side-by-side the normalized test set and the one-hot encoded features
    X_test_std = pd.concat([X_test_std, X_test[ohe_features]], axis=1)
    return X_train_std, X_test_std

X_train_std, X_test_std = train_test_normalize(X_train, X_test)

In [7]:
# Convert train/test sets to numpy float32 ndarrays
# (after doing some research it appears that float32 should be used when using a GPU)
X_train_std = X_train_std.to_numpy().astype(np.float32)
X_test_std = X_test_std.to_numpy().astype(np.float32)
y_train = y_train.to_numpy().astype(np.float32)
y_test = y_test.to_numpy().astype(np.float32)

In [8]:
# Initialize the training/validation set splitter.
# We'll use it later for the grid-search, performing a 10-fold cross validation
# where each validation set is 25% the size of the training set.
# This yields 1620 samples per class for the new training set and
# 540 samples per class for each validation set.
validation_strat = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=10)

In [9]:
# Subclass KerasClassifier and override fit() method
# to fix OOM error when running sklearn GridSearchCV on the GPU
class GridKerasClassifier(keras.wrappers.scikit_learn.KerasClassifier):
    def fit(self, *args, **kwargs):
        # Clear tensorflow session each time fit is invoked
        keras.backend.clear_session()
        # Use custom configuration for the new session
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        session = tf.Session(config=config)
        keras.backend.set_session(session)
        super().fit(*args, **kwargs)

# Wrapper function which builds the classifier architecture
def make_model(n_features=51, n_classes=7, dense_layer_size=120, learning_rate=0.5):
    model = keras.Sequential()
    model.add(keras.layers.Dense(dense_layer_size, activation='relu', input_dim=n_features))
    model.add(keras.layers.Dense(n_classes, activation='softmax'))
    sgd = keras.optimizers.SGD(lr=learning_rate) # per il momentum: momentum=0.9
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [10]:
# Wrap the Keras neural network onto a scikit Classifier
classifier_net = GridKerasClassifier(make_model)

# For params tuning see
# https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

# Define the grid search params
# uncomment for changing the default number of features and/or classes
# for the network. DON'T FORGET TO CHANGE THE TRAINING/TEST SET ACCORDINGLY!
# n_features = [X_train_std.shape[1]]
# n_classes = [y_train.shape[1]]
dense_layer_size = [120]
# 0.5 is the best so far
learning_rate = [0.5]
# 128 seems to work well
batch_size = [128]
# Try 100, 500, 1000
epochs = [10]
# Define the grid of parameters 
param_grid = dict(# n_features=n_features, # uncomment if needed
                  # n_classes=n_classes,   # uncomment if needed
                  dense_layer_size=dense_layer_size,
                  learning_rate=learning_rate,
                  batch_size=batch_size,
                  epochs=epochs)
# Initialize the grid search using the nn classifier and the cross-validation
# strategy defined above. Since clear_session() is invoked after each CV run,
# it's fine to set n_jobs=-1 when running on the GPU.
grid = GridSearchCV(estimator=classifier_net,
                    param_grid=param_grid,
                    n_jobs=-1,
                    cv=validation_strat,
                    verbose=2)

In [11]:
# Run grid search
# THIS CELL IS TIME CONSUMING, BE AWARE OF THAT!
grid.fit(X_train_std, y_train)
# Store the best model in the current dir
#now = datetime.now()
#timestamp = now.strftime('%b-%d-%Y_%H%M')
#best_model_fname = f'hidden{dense_layer_size}_epochs{epochs}_{timestamp}.hdf5'
#best_model_path = os.path.join(os.getcwd(), best_model_fname)
#keras.models.save_model(model=grid.best_estimator_.model,
#                        filepath=best_model_path,
#                        save_format='h5')

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   21.8s remaining:   21.8s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   21.9s finished
W0715 07:41:31.182157 140080279652160 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Epoch 1/10
15120/15120 [==============================] - 0s 29us/sample - loss: 0.9379 - acc: 0.6284
Epoch 2/10
15120/15120 [==============================] - 0s 16us/sample - loss: 0.7166 - acc: 0.6980
Epoch 3/10
15120/15120 [==============================] - 0s 16us/sample - loss: 0.6723 - acc: 0.7168
Epoch 4/10
15120/15120 [==============================] - 0s 15us/sample - loss: 0.6410 - acc: 0.7309
Epoch 5/10
15120/15120 [==============================] - 0s 16us/sample - loss: 0.6214 - acc: 0.7410
Epoch 6/10
15120/15120 [==============================] - 0s 16us/sample - loss: 0.6056 - acc: 0.7438
Epoch 7/10
15120/15120 [==============================] - 0s 16us/sample - loss: 0.5891 - acc: 0.7563
Epoch 8/10
15120/15120 [==============================] - 0s 15us/sample - loss: 0.5798 - acc: 0.7571
Epoch 9/10
15120/15120 [==============================] - 0s 16us/sample - loss: 0.5748 - acc: 0.7620
Epoch 10/10
15120/15120 [==============================] - 0s 17us/sample - loss: 

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=10, test_size=0.25,
            train_size=None),
             error_score='raise-deprecating',
             estimator=<__main__.GridKerasClassifier object at 0x7f66f006cd30>,
             iid='warn', n_jobs=-1,
             param_grid={'batch_size': [128], 'dense_layer_size': [120],
                         'epochs': [10], 'learning_rate': [0.5]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=2)

In [12]:
# Print the best scores defined earlier for the grid search and its model params
print(f'Best scores: {grid.best_score_:.4f} using {grid.best_params_}', end='\n\n')
# Get the mean of each score for each cross-validation run
means = grid.cv_results_['mean_test_score']
# Get the standard dev of each score for each cross-validation run
stds = grid.cv_results_['std_test_score']
# Get the model params for each cross-validation run
params = grid.cv_results_['params']
# Loop through means, stds, params and print
# one line for each cross-validation run
for mean, stdev, param in zip(means, stds, params):
    print(f'Mean: {mean:.4f} +- {stdev:.6f} with: {param}')

Best scores: 0.7458 using {'batch_size': 128, 'dense_layer_size': 120, 'epochs': 10, 'learning_rate': 0.5}

Mean: 0.7458 +- 0.013395 with: {'batch_size': 128, 'dense_layer_size': 120, 'epochs': 10, 'learning_rate': 0.5}
